Trabalho trilha 06 DeepLearning

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ion.csv to ion.csv


In [ ]:
data = pd.read_csv('ion.csv')
data.head()

,Unnamed: 0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,Class
0,1,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,good
1,2,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,bad
2,3,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,good
3,4,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,bad
4,5,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,good


# Pré processamento


In [ ]:
data['Class'] = data['Class'].map({'good': 0, 'bad': 1})

In [ ]:
X = data.values[:, :-1]
y = data.values[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

In [ ]:
iterations = 100
batch_size = 33

## Data Pipeline

In [ ]:
from torch.utils.data import Dataset

class TrainData(Dataset):    
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
    
    def __getitem__(self, index):
        return self.x_train[index], self.y_train[index]
    
    def __len__ (self):
        return len(self.x_train)

    
class TestData(Dataset):    
    def __init__(self, x_test):
        self.x_test = x_test
        
    def __getitem__(self, index):
        return self.x_test[index]
        
    def __len__ (self):
        return len(self.x_test)

In [ ]:
train_data = TrainData(torch.from_numpy(x_train).to(torch.float32), torch.from_numpy(y_train).to(torch.float32))
test_data = TestData(torch.from_numpy(x_test).to(torch.float32))

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

## Definição do modelo


In [ ]:
class ModeloNils(nn.Module):    
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(35, 64)
        self.linear_2 = nn.Linear(64, 64)
        self.linear_3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.1)
    
    def forward(self, inputs):
        out = self.linear_1(inputs)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        out = self.dropout(out)
        out = self.linear_3(out)
        return out

## Treinamento

In [ ]:
model = ModeloNils()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
from sklearn.metrics import roc_auc_score

n_batches = len(train_loader)
model.train()
loss_li = []
score_li = []

for it in range(iterations):
    it_loss = 0
    it_score = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_real = y_batch.unsqueeze(1)
        print(x_batch)
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_real)
        y_pred = torch.sigmoid(y_pred.detach())
        score = roc_auc_score(y_real, y_pred)
        acc = (y_pred == y_real).sum().item()
        acc = acc / batch_size
        loss.backward()
        optimizer.step()
        batch_history["loss"].append(loss.item())
        batch_history["accuracy"].append(acc)

        it_loss += loss.item()
        it_score += score
    loss_li.append(it_loss / n_batches)
    score_li.append(it_score / n_batches)
        

A saída de streaming foi truncada nas últimas 5000 linhas.
         -6.1446e-01, -2.6389e-01, -7.0835e-01, -8.9370e-02, -7.1273e-01],
        [ 3.4000e+02,  1.0000e+00,  0.0000e+00,  6.5845e-01,  4.3617e-01,
          4.4681e-01,  7.4804e-01,  5.3190e-02,  8.5106e-01, -3.2027e-01,
          8.2139e-01, -6.8253e-01,  5.2408e-01, -8.4211e-01,  7.1110e-02,
         -8.2811e-01, -2.8723e-01, -4.7032e-01, -7.1725e-01, -4.7590e-02,
         -8.6002e-01,  2.3292e-01, -7.6316e-01,  5.6663e-01, -5.2128e-01,
          7.4300e-01, -1.8645e-01,  7.4758e-01,  2.3713e-01,  4.5185e-01,
          5.9071e-01,  2.0549e-01,  7.6764e-01, -1.8533e-01,  7.4356e-01]])
[Iteration 76] Loss: 0.0276, Area-Under-Curve: 1.0000
tensor([[ 1.0400e+02,  1.0000e+00,  0.0000e+00,  ..., -3.1857e-01,
          6.9531e-01, -4.1573e-01],
        [ 2.8300e+02,  1.0000e+00,  0.0000e+00,  ..., -7.0835e-01,
         -8.9370e-02, -7.1273e-01],
        [ 1.2000e+02,  1.0000e+00,  0.0000e+00,  ..., -1.5609e-01,
          1.0000e+0

## Avaliação do model, matris de confusão, precision, recall e F1 score

In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for x_batch in test_loader:
        y_pred = model(x_batch)
        y_pred = torch.sigmoid(y_pred)
        predictions.append(y_pred.squeeze().tolist())

y_pred = np.round(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[48,  0],
       [ 5, 18]])

In [ ]:
from sklearn.metrics import classification_report

metrics = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
metrics

{'0.0': {'f1-score': 0.9504950495049505,
  'precision': 0.9056603773584906,
  'recall': 1.0,
  'support': 48},
 '1.0': {'f1-score': 0.878048780487805,
  'precision': 1.0,
  'recall': 0.782608695652174,
  'support': 23},
 'accuracy': 0.9295774647887324,
 'macro avg': {'f1-score': 0.9142719149963777,
  'precision': 0.9528301886792453,
  'recall': 0.8913043478260869,
  'support': 71},
 'weighted avg': {'f1-score': 0.92702653982334,
  'precision': 0.9362211001860219,
  'recall': 0.9295774647887324,
  'support': 71}}

In [ ]:
metrics['accuracy']

0.9295774647887324